# CELL 0 — Thiết lập biến môi trường

## Mục đích
Cell này thiết lập các biến môi trường **TRƯỚC KHI** import các thư viện để tránh các lỗi xung đột.

## Giải thích chi tiết các biến:

| Biến môi trường | Giá trị | Mục đích |
|-----------------|---------|----------|
| `TRANSFORMERS_NO_TF` | "1" | Chặn Transformers tự động import TensorFlow |
| `TRANSFORMERS_NO_FLAX` | "1" | Chặn Transformers tự động import Flax |
| `TF_CPP_MIN_LOG_LEVEL` | "3" | Tắt các log warning của TensorFlow C++ backend |
| `TOKENIZERS_PARALLELISM` | "false" | Tắt tokenization song song để tránh deadlock |

In [1]:
import os

# Chặn Transformers tự động import TensorFlow/Flax (tránh lỗi protobuf / MessageFactory)
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


print("[OK] Environment flags set (no TF / no Flax).")


[OK] Environment flags set (no TF / no Flax).


# CELL 1 — Kiểm tra môi trường và GPU

## Mục đích
Kiểm tra và hiển thị thông tin về môi trường runtime để đảm bảo:
- Phiên bản Python và các thư viện tương thích
- GPU có sẵn để tăng tốc training

## Các thông tin được kiểm tra:

| Thông tin | Mô tả |
|-----------|-------|
| `sys.version` | Phiên bản Python đang chạy |
| `torch.__version__` | Phiên bản PyTorch |
| `torch.cuda.is_available()` | Kiểm tra CUDA có sẵn không |
| `torch.cuda.get_device_name(0)` | Tên GPU (nếu có) |
| `transformers.__version__` | Phiên bản Hugging Face Transformers |
| `datasets.__version__` | Phiên bản Hugging Face Datasets |
| `accelerate.__version__` | Phiên bản Accelerate (hỗ trợ distributed training) |


In [2]:
import sys, torch
import numpy as np

import transformers, datasets, accelerate

print("Python:", sys.version.split()[0])
print("Torch:", torch.__version__)
print("CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)
print("Accelerate:", accelerate.__version__)


Python: 3.11.13
Torch: 2.6.0+cu124
CUDA: True
GPU: Tesla T4
Transformers: 4.53.3
Datasets: 4.4.1
Accelerate: 1.9.0


# CELL 2 — Cài đặt các package bổ sung

## Mục đích
Cài đặt các package cần thiết nếu chưa có trong môi trường.

## Các package được cài:

| Package | Mục đích | Bắt buộc? |
|---------|----------|-----------|
| `sacrebleu` | Tính điểm BLEU chuẩn cho đánh giá dịch máy | ✅ Bắt buộc |
| `hf_transfer` | Tăng tốc download/upload model từ Hugging Face Hub | ⚪ Tùy chọn |

## Về sacrebleu
- **SacreBLEU** là công cụ chuẩn để đánh giá chất lượng dịch máy
- Tính điểm BLEU (Bilingual Evaluation Understudy) từ 0-100
- Điểm càng cao = bản dịch càng giống với reference

In [3]:
import importlib.util, subprocess

def pip_install(pkg: str):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

# sacrebleu: bắt buộc
if importlib.util.find_spec("sacrebleu") is None:
    print("[INFO] Installing sacrebleu ...")
    pip_install("sacrebleu")
else:
    print("[OK] sacrebleu already installed")

# hf_transfer: optional tăng tốc push/pull
if importlib.util.find_spec("hf_transfer") is None:
    print("[INFO] Installing hf_transfer (optional) ...")
    pip_install("hf_transfer")
else:
    print("[OK] hf_transfer already installed")


[INFO] Installing sacrebleu ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.5 MB/s eta 0:00:00
[OK] hf_transfer already installed


# CELL 3 — Import các thư viện chính

## Mục đích
Import tất cả các thư viện cần thiết cho pipeline dịch máy.

## Các thư viện được import:

### Thư viện chuẩn Python


### Hugging Face Libraries 
Mục đích: nơi lưu trữ trạng thái và thông số sau khi train model


### Evaluation

| `sacrebleu` | Tính điểm BLEU chuẩn |


In [4]:
import re, math, random, hashlib
from pathlib import Path
from typing import List, Dict, Any

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
import sacrebleu

print("[OK] Imports done.")


E0000 00:00:1765738492.522072      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765738492.625769      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

[OK] Imports done.


# CELL 4 — Tự động tìm thư mục dataset

## Mục đích
Tự động dò tìm thư mục chứa dataset trên Kaggle mà không cần hardcode đường dẫn.

## Các file dataset cần có:

| File | Nội dung |
|------|----------|
| `train.en.txt` | Câu tiếng Anh để training |
| `train.vi.txt` | Câu tiếng Việt tương ứng (parallel corpus) |
| `public_test.en.txt` | Câu tiếng Anh để test |
| `public_test.vi.txt` | Câu tiếng Việt reference để tính BLEU |


In [5]:
from pathlib import Path

def find_dataset_dir() -> Path:
    base = Path("/kaggle/input")
    for d in base.glob("*"):
        if d.is_dir() and (d / "train.en.txt").exists() and (d / "train.vi.txt").exists():
            return d
    for d in base.glob("*/*"):
        if d.is_dir() and (d / "train.en.txt").exists() and (d / "train.vi.txt").exists():
            return d
    raise FileNotFoundError("Could not find dataset folder containing train.en.txt/train.vi.txt")

DATA_DIR = find_dataset_dir()
print("[OK] Found dataset dir:", DATA_DIR)

TRAIN_EN = DATA_DIR / "train.en.txt"
TRAIN_VI = DATA_DIR / "train.vi.txt"
TEST_EN  = DATA_DIR / "public_test.en.txt"
TEST_VI  = DATA_DIR / "public_test.vi.txt"

for p in [TRAIN_EN, TRAIN_VI, TEST_EN, TEST_VI]:
    print("[CHECK]", p.name, "->", "OK" if p.exists() else "NOT FOUND")


[OK] Found dataset dir: /kaggle/input/databaitoanphu
[CHECK] train.en.txt -> OK
[CHECK] train.vi.txt -> OK
[CHECK] public_test.en.txt -> OK
[CHECK] public_test.vi.txt -> OK


# CELL 5 — Đọc file song song EN/VI

## Mục đích
Đọc và ghép cặp các câu tiếng Anh - tiếng Việt từ file text.

## Các hàm được định nghĩa:

### `read_lines(path, encoding="utf-8")`

### `load_parallel(en_path, vi_path, name="data")`

## Parallel Corpus là gì?
- Là tập dữ liệu gồm các cặp câu song ngữ (EN-VI) được align theo từng dòng
- Dòng 1 file EN ↔ Dòng 1 file VI
- Dòng 2 file EN ↔ Dòng 2 file VI

In [6]:
def read_lines(path: Path, encoding="utf-8"):
    with open(path, "r", encoding=encoding, errors="replace") as f:
        return [line.rstrip("\n") for line in f]

def load_parallel(en_path: Path, vi_path: Path, name="data"):
    en_lines = read_lines(en_path)
    vi_lines = read_lines(vi_path)
    n = min(len(en_lines), len(vi_lines))
    if len(en_lines) != len(vi_lines):
        print(f"[WARN] {name} mismatch -> EN={len(en_lines):,}, VI={len(vi_lines):,}. Using first {n:,} pairs.")
    else:
        print(f"[INFO] {name} lines -> {n:,} pairs.")
    return list(zip(en_lines[:n], vi_lines[:n]))

train_pairs_raw = load_parallel(TRAIN_EN, TRAIN_VI, "train_raw")
test_pairs_raw  = load_parallel(TEST_EN,  TEST_VI,  "test_raw")

print("[SAMPLE train_raw]", train_pairs_raw[0])
print("[SAMPLE test_raw ]", test_pairs_raw[0])


[INFO] train_raw lines -> 500,000 pairs.
[INFO] test_raw lines -> 3,000 pairs.
[SAMPLE train_raw] ('To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital', 'Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.A tại Khoa Tai mũi họng - Bệnh viện Trung ương Thái Nguyên')
[SAMPLE test_raw ] ('Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao', 'Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017')


#  CELL 6 — Làm sạch, Lọc và Loại bỏ trùng lặp

## Mục đích
Tiền xử lý dữ liệu để đảm bảo chất lượng training data.

## Ba bước xử lý:

### 1. Clean (Làm sạch) - Hàm `basic_clean(s)`
```python
s.strip()                    # Xóa whitespace đầu/cuối
re.sub(r"\s+", " ", s)       # Chuẩn hóa nhiều space thành 1 space
```

### 2. Filter (Lọc) - Hàm `is_good_pair(src, tgt)`
| Điều kiện | Giá trị | Mục đích |
|-----------|---------|----------|
| `MIN_CHARS` | 2 | Loại câu quá ngắn (noise) |
| `MAX_CHARS` | 400 | Loại câu quá dài (tránh truncate quá nhiều) |
| `not empty` | - | Loại cặp có câu rỗng |

### 3. Dedup (Loại bỏ trùng lặp)
- Tạo **MD5 hash** từ `src + "\t" + tgt`
- Dùng `set()` để track các hash đã thấy
- Chỉ giữ cặp có hash chưa xuất hiện

In [7]:
import re, hashlib

def basic_clean(s: str) -> str:
    s = "" if s is None else s
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s

MIN_CHARS = 2
MAX_CHARS = 400

def is_good_pair(src: str, tgt: str) -> bool:
    if not src or not tgt:
        return False
    if len(src) < MIN_CHARS or len(tgt) < MIN_CHARS:
        return False
    if len(src) > MAX_CHARS or len(tgt) > MAX_CHARS:
        return False
    return True

def clean_filter_dedup(pairs, name="data"):
    # clean + filter
    cleaned = []
    for src, tgt in pairs:
        src = basic_clean(src)
        tgt = basic_clean(tgt)
        if is_good_pair(src, tgt):
            cleaned.append((src, tgt))
    print(f"[INFO] {name}: after clean+filter -> {len(cleaned):,} pairs")

    # dedup
    seen = set()
    dedup = []
    for src, tgt in cleaned:
        h = hashlib.md5((src + "\t" + tgt).encode("utf-8")).hexdigest()
        if h not in seen:
            seen.add(h)
            dedup.append((src, tgt))
    print(f"[INFO] {name}: after dedup       -> {len(dedup):,} pairs")
    return dedup

train_pairs = clean_filter_dedup(train_pairs_raw, "train")
test_pairs  = clean_filter_dedup(test_pairs_raw,  "test")

print("[SAMPLE clean train]", train_pairs[0])
print("[SAMPLE clean test ]", test_pairs[0])


[INFO] train: after clean+filter -> 489,008 pairs
[INFO] train: after dedup       -> 340,522 pairs
[INFO] test: after clean+filter -> 2,946 pairs
[INFO] test: after dedup       -> 2,943 pairs
[SAMPLE clean train] ('To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital', 'Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.A tại Khoa Tai mũi họng - Bệnh viện Trung ương Thái Nguyên')
[SAMPLE clean test ] ('Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao', 'Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017')


# CELL 7 — Sampling và Chia Train/Validation/Test

## Mục đích
Tạo dataset với kích thước cố định và chia thành các tập train/validation/test.

## Các tham số:

| Tham số | Giá trị | Mô tả |
|---------|---------|-------|
| `SEED` | 42 | Seed cho reproducibility (kết quả giống nhau mỗi lần chạy) |
| `N_TOTAL` | 20,000 | Số cặp câu lấy từ train_pairs |
| `TRAIN_RATIO` | 0.9 | 90% cho train, 10% cho validation |

| train | 18,000 | 90% từ subset |
| validation | 2,000 | 10% từ subset |
| test | 2,943 | public_test (giữ nguyên) |

In [8]:
import random
from datasets import Dataset, DatasetDict

SEED = 42
N_TOTAL = 20000

TRAIN_RATIO = 0.9  # 90/10

# 1) shuffle train_pairs
rng = random.Random(SEED)
rng.shuffle(train_pairs)

# 2) lấy 5000 (nếu không đủ thì lấy hết)
n_take = min(N_TOTAL, len(train_pairs))
subset = train_pairs[:n_take]
print(f"[INFO] Train pairs available: {len(train_pairs):,}")
print(f"[INFO] Taking from train     : {n_take:,} pairs")

# 3) split 90/10
n_train = int(n_take * TRAIN_RATIO)
n_val = n_take - n_train

train_subset = subset[:n_train]
val_subset   = subset[n_train:]

print(f"[INFO] Split -> train={len(train_subset):,} | val={len(val_subset):,}")

# 4) build datasets
train_ds = Dataset.from_dict({
    "src_text": [s for s, _ in train_subset],
    "tgt_text": [t for _, t in train_subset],
})
val_ds = Dataset.from_dict({
    "src_text": [s for s, _ in val_subset],
    "tgt_text": [t for _, t in val_subset],
})

# test giữ nguyên từ public_test
test_ds = Dataset.from_dict({
    "src_text": [s for s, _ in test_pairs],
    "tgt_text": [t for _, t in test_pairs],
})

ds = DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})
print(ds)

print("[SAMPLE TRAIN]", ds["train"][0])
print("[SAMPLE VAL  ]", ds["validation"][0])
print("[SAMPLE TEST ]", ds["test"][0])


[INFO] Train pairs available: 340,522
[INFO] Taking from train     : 20,000 pairs
[INFO] Split -> train=18,000 | val=2,000
DatasetDict({
    train: Dataset({
        features: ['src_text', 'tgt_text'],
        num_rows: 18000
    })
    validation: Dataset({
        features: ['src_text', 'tgt_text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['src_text', 'tgt_text'],
        num_rows: 2943
    })
})
[SAMPLE TRAIN] {'src_text': 'Monitor and control ICP using sedatives, endotracheal intubation, hyperventilation, hydration, diuretics, measures to control blood pressure, and sometimes corticosteroids.', 'tgt_text': 'Theo dõi và kiểm soát ICP bằng cách sử dụng thuốc an thần, đặt nội khí quản, tăng thông khí, hydrat hoá, thuốc lợi tiểu, các biện pháp để kiểm soát huyết áp, và đôi khi là corticosteroid.'}
[SAMPLE VAL  ] {'src_text': 'The menstrual cycle is one of the most common acne triggers.', 'tgt_text': 'Chu kỳ kinh nguyệt là một trong những nguyên nhân thường g

# CELL 8a — Load Tokenizer và Model mBART-50

## Mục đích
Load model mBART-50 pre-trained từ Hugging Face Hub để fine-tune cho task dịch EN→VI.

## Về mBART-50

### Model ID
```python
MODEL_ID = "facebook/mbart-large-50-many-to-many-mmt"
```

### Đặc điểm:
| Thuộc tính | Giá trị |
|------------|---------|
| Kiến trúc | Transformer Encoder-Decoder |
| Số ngôn ngữ | 50 ngôn ngữ |
| Số parameters | ~611M |
| Pre-training | Denoising auto-encoding trên 50 ngôn ngữ |
| Fine-tuning | Many-to-many machine translation |

## Cấu hình ngôn ngữ

| Biến | Giá trị | Ý nghĩa |
|------|---------|---------|
| `SRC_LANG` | "en_XX" | Mã ngôn ngữ nguồn (English) |
| `TGT_LANG` | "vi_VN" | Mã ngôn ngữ đích (Vietnamese) |

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

MODEL_ID = "facebook/mbart-large-50-many-to-many-mmt"
SRC_LANG = "en_XX"
TGT_LANG = "vi_VN"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

# set src/tgt language (QUAN TRỌNG)
tokenizer.src_lang = SRC_LANG
tokenizer.tgt_lang = TGT_LANG   # <-- FIX KeyError: None

forced_bos_token_id = tokenizer.lang_code_to_id[TGT_LANG]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("[OK] tokenizer/model loaded")
print("[INFO] tokenizer.src_lang:", tokenizer.src_lang)
print("[INFO] tokenizer.tgt_lang:", tokenizer.tgt_lang)
print("[INFO] forced_bos_token_id:", forced_bos_token_id)


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

[OK] tokenizer/model loaded
[INFO] tokenizer.src_lang: en_XX
[INFO] tokenizer.tgt_lang: vi_VN
[INFO] forced_bos_token_id: 250024


# CELL 8 — Tokenize Dataset

## Mục đích
Chuyển đổi text thô thành token IDs để model có thể xử lý.

## Tham số Tokenization:

| Tham số | Giá trị | Mô tả |
|---------|---------|-------|
| `MAX_SRC_LEN` | 256 | Độ dài tối đa source (EN) |
| `MAX_TGT_LEN` | 256 | Độ dài tối đa target (VI) |

In [10]:
# đảm bảo ds đã có ở cell split 5000 (train/validation/test)
assert "ds" in globals(), "You must run the data-loading/splitting cell to create `ds` first!"
assert "tokenizer" in globals(), "You must run the tokenizer/model loading cell first!"

MAX_SRC_LEN = 256
MAX_TGT_LEN = 256

def preprocess_batch(batch):
    tokenizer.src_lang = SRC_LANG

    model_inputs = tokenizer(
        batch["src_text"],
        max_length=MAX_SRC_LEN,
        truncation=True,
        padding=False,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["tgt_text"],
            max_length=MAX_TGT_LEN,
            truncation=True,
            padding=False,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tok = ds["train"].map(preprocess_batch, batched=True, remove_columns=ds["train"].column_names)
val_tok   = ds["validation"].map(preprocess_batch, batched=True, remove_columns=ds["validation"].column_names)
test_tok  = ds["test"].map(preprocess_batch, batched=True, remove_columns=ds["test"].column_names)

print("[OK] Tokenized sizes:",
      "train", len(train_tok),
      "| val", len(val_tok),
      "| test", len(test_tok))


Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2943 [00:00<?, ? examples/s]

[OK] Tokenized sizes: train 18000 | val 2000 | test 2943


# CELL 9 — Đánh giá BLEU trước khi Training (Baseline)

## Mục đích
Đo điểm BLEU của model **TRƯỚC** fine-tuning để có baseline so sánh.

## Hàm `translate_en2vi(texts, ...)`

### Tham số Generation:
| Tham số | Giá trị | Mô tả |
|---------|---------|-------|
| `num_beams` | 5 | Beam search với 5 beams |
| `length_penalty` | 1.0 | Không penalty độ dài (1.0 = neutral) |
| `no_repeat_ngram_size` | 3 | Không cho phép lặp trigram |
| `max_new_tokens` | 128 | Tối đa 128 tokens output |
| `early_stopping` | True | Dừng khi tất cả beams đạt EOS |

In [11]:
@torch.inference_mode()
def translate_en2vi(texts, num_beams=5, length_penalty=1.0, no_repeat_ngram_size=3, max_new_tokens=128):
    model.eval()
    tokenizer.src_lang = SRC_LANG
    enc = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_SRC_LEN).to(model.device)
    out = model.generate(
        **enc,
        forced_bos_token_id=tokenizer.lang_code_to_id[TGT_LANG],
        num_beams=num_beams,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        max_new_tokens=max_new_tokens,
        early_stopping=True
    )
    return tokenizer.batch_decode(out, skip_special_tokens=True)

def bleu_on_test(max_samples=None):
    srcs = ds["test"]["src_text"]
    refs = ds["test"]["tgt_text"]

    if max_samples is not None:
        srcs = srcs[:max_samples]
        refs = refs[:max_samples]

    hyps = []
    bs = 16 if torch.cuda.is_available() else 4
    for i in range(0, len(srcs), bs):
        hyps.extend(translate_en2vi(srcs[i:i+bs]))

    bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
    print(f"[TEST] sacreBLEU ({len(srcs)} samples): {bleu:.2f}")
    return bleu

_ = bleu_on_test(max_samples=200)   # đổi None để chạy full public_test


[TEST] sacreBLEU (200 samples): 26.20


# CELL — Đăng nhập Hugging Face Hub

## Mục đích
Xác thực với Hugging Face để có thể push model lên Hub sau khi training.

## Cách lấy Token trên Kaggle

### Bước 1: Tạo HF Token
1. Vào https://huggingface.co/settings/tokens
2. Tạo token với quyền `write`

### Bước 2: Lưu vào Kaggle Secrets
### Bước 3: Sử dụng trong code


In [12]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()

# tên secret của anh là HF_TOKEN (đúng như panel)
hf_token = user_secrets.get_secret("HF_TOKEN")

login(token=hf_token)
print("[OK] Logged in to Hugging Face.")


[OK] Logged in to Hugging Face.


Load model đã train sẵn từ trước trên hugging face về làm model train tiếp data mới 


In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

HF_REPO = "ngothuyet/mbart50-envi"

tokenizer = AutoTokenizer.from_pretrained(HF_REPO, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(HF_REPO)

tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "vi_VN"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("[OK] Loaded model from HF:", HF_REPO)


tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

[OK] Loaded model from HF: ngothuyet/mbart50-envi


Đọc train, val loss từ model trước đó

In [14]:
import pandas as pd
from huggingface_hub import hf_hub_download

HF_REPO_ID = "ngothuyet/mbart50-envi"   # repo của anh
FILENAME = "train_val_loss.tsv"

path = hf_hub_download(repo_id=HF_REPO_ID, filename=FILENAME)
df = pd.read_csv(path, sep="\t")

print("===== ALL TRAIN/VAL LOSS (FROM HF) =====")
print(df.to_string(index=False))   # in hết bảng


train_val_loss.tsv:   0%|          | 0.00/114 [00:00<?, ?B/s]

===== ALL TRAIN/VAL LOSS (FROM HF) =====
 epoch  train_loss  val_loss
   1.0      1.8957  1.658935
   2.0      1.3897  1.586371
   3.0      1.1786  1.592958


In [15]:
import os, inspect
import torch
import pandas as pd
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

# ====== CHECK ======
assert "model" in globals(), "Missing `model`"
assert "tokenizer" in globals(), "Missing `tokenizer`"
assert "train_tok" in globals(), "Missing `train_tok`"
assert "val_tok" in globals(), "Missing `val_tok`"
assert "SRC_LANG" in globals() and "TGT_LANG" in globals(), "Missing SRC_LANG/TGT_LANG"

# Fix stateful tokenizer for mBART-50
tokenizer.src_lang = SRC_LANG
tokenizer.tgt_lang = TGT_LANG

# ====== CONFIG (anh chỉnh ở đây) ======
OUTPUT_DIR  = "mbart50_envi"
NUM_EPOCHS  = 3

TRAIN_BS = 2
EVAL_BS  = 2
GRAD_ACC = 8
LR       = 3e-5

# Hugging Face repo
HF_REPO_ID = "ngothuyet/mbart50-envi"
HF_PRIVATE = True

# ====== DATA COLLATOR ======
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# ====== TrainingArguments ======
# Mục tiêu: eval/log theo epoch để có loss, nhưng KHÔNG save checkpoint.
ta_kwargs = dict(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,

    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BS,
    per_device_eval_batch_size=EVAL_BS,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    warmup_ratio=0.03,
    weight_decay=0.01,

    # ✅ log + eval theo epoch để lấy train_loss & val_loss
    predict_with_generate=False,     # chỉ cần loss -> nhanh
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    report_to="none",

    # ✅ KHÔNG LƯU CHECKPOINT (tránh lỗi write optimizer.pt)
    save_strategy="no",              # ❌ không checkpoint
    save_only_model=True,            # ❌ không optimizer/scheduler (an toàn)
)

sig = inspect.signature(Seq2SeqTrainingArguments.__init__).parameters

# logging theo epoch
if "logging_strategy" in sig:
    ta_kwargs["logging_strategy"] = "epoch"

# evaluation theo epoch (tên field có thể khác nhau theo version)
if "evaluation_strategy" in sig:
    ta_kwargs["evaluation_strategy"] = "epoch"
elif "eval_strategy" in sig:
    ta_kwargs["eval_strategy"] = "epoch"
else:
    raise ValueError("Transformers version: missing evaluation_strategy/eval_strategy")

# ✅ Push chỉ 1 lần ở cuối (vì không save checkpoint)
# (không dùng hub_strategy="every_save" vì không còn save)
ta_kwargs.update(dict(
    push_to_hub=True,
    hub_model_id=HF_REPO_ID,
    hub_private_repo=HF_PRIVATE,
    hub_strategy="end",              # ✅ chỉ push cuối
))

training_args = Seq2SeqTrainingArguments(**ta_kwargs)

# ====== TRAINER ======
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("[INFO] Start training...")
train_result = trainer.train()
print("\n===== TRAIN DONE =====")
print(train_result)

print("\n===== FINAL EVAL (VAL) =====")
final_metrics = trainer.evaluate()
for k, v in final_metrics.items():
    print(f"{k}: {v}")

# ====== SAVE FINAL LOCAL (model + tokenizer) ======
trainer.save_model(OUTPUT_DIR)          # tạo model.safetensors, config.json,...
tokenizer.save_pretrained(OUTPUT_DIR)   # tokenizer files
print("[OK] Saved final model/tokenizer to:", OUTPUT_DIR)

# ====== EXPORT TSV (train_loss + val_loss) ======
logs = trainer.state.log_history

train_loss_by_epoch = {}
val_loss_by_epoch = {}

for row in logs:
    # train loss rows
    if "epoch" in row and "loss" in row and "eval_loss" not in row:
        train_loss_by_epoch[row["epoch"]] = row["loss"]
    # eval loss rows
    if "epoch" in row and "eval_loss" in row:
        val_loss_by_epoch[row["epoch"]] = row["eval_loss"]

epochs = sorted(set(list(train_loss_by_epoch.keys()) + list(val_loss_by_epoch.keys())))
df = pd.DataFrame({
    "epoch": epochs,
    "train_loss": [train_loss_by_epoch.get(e, None) for e in epochs],
    "val_loss": [val_loss_by_epoch.get(e, None) for e in epochs],
})

tsv_path = os.path.join(OUTPUT_DIR, "train_val_loss.tsv")
df.to_csv(tsv_path, sep="\t", index=False)
print("[OK] Saved TSV:", tsv_path)
display(df)

# ====== PUSH FINAL + TSV ======
# Vì OUTPUT_DIR chứa model + tokenizer + training_args.bin + train_val_loss.tsv
trainer.push_to_hub(commit_message="Final after training (no checkpoints) + train_val_loss.tsv")
print("[OK] Pushed final model + TSV to Hugging Face.")


[INFO] Start training...


/tmp/ipykernel_47/3259613353.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.500800,1.360325
2,1.171500,1.301108
3,0.991700,1.295173



===== TRAIN DONE =====
TrainOutput(global_step=1689, training_loss=1.2213178129856794, metrics={'train_runtime': 10354.0679, 'train_samples_per_second': 5.215, 'train_steps_per_second': 0.163, 'total_flos': 6193749995618304.0, 'train_loss': 1.2213178129856794, 'epoch': 3.0})

===== FINAL EVAL (VAL) =====


eval_loss: 1.2951726913452148
eval_runtime: 153.5677
eval_samples_per_second: 13.024
eval_steps_per_second: 3.256
epoch: 3.0


...

[OK] Saved final model/tokenizer to: mbart50_envi
[OK] Saved TSV: mbart50_envi/train_val_loss.tsv


,epoch,train_loss,val_loss
0,1.0,1.5008,1.360325
1,2.0,1.1715,1.301108
2,3.0,0.9917,1.295173


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

[OK] Pushed final model + TSV to Hugging Face.


In [16]:
# lấy log_history và gom theo epoch
logs = trainer.state.log_history

train_loss_by_epoch = {}
val_loss_by_epoch = {}

for row in logs:
    if "epoch" in row and "loss" in row and "eval_loss" not in row:
        train_loss_by_epoch[row["epoch"]] = row["loss"]
    if "epoch" in row and "eval_loss" in row:
        val_loss_by_epoch[row["epoch"]] = row["eval_loss"]

epochs = sorted(set(list(train_loss_by_epoch.keys()) + list(val_loss_by_epoch.keys())))

df = pd.DataFrame({
    "epoch": epochs,
    "train_loss": [train_loss_by_epoch.get(e, None) for e in epochs],
    "val_loss": [val_loss_by_epoch.get(e, None) for e in epochs],
})

tsv_path = f"{OUTPUT_DIR}/train_val_loss.tsv"
df.to_csv(tsv_path, sep="\t", index=False)

print("[OK] Saved TSV:", tsv_path)
display(df)

# push TSV lên hub (nằm trong output_dir sẽ được push)
trainer.push_to_hub(commit_message="Add train_val_loss.tsv")
print("[OK] Pushed TSV to hub.")


[OK] Saved TSV: mbart50_envi/train_val_loss.tsv


,epoch,train_loss,val_loss
0,1.0,1.5008,1.360325
1,2.0,1.1715,1.301108
2,3.0,0.9917,1.295173


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


[OK] Pushed TSV to hub.


Load model từ hugging face 

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

HF_REPO_ID = "ngothuyet/mbart50-envi"  # <-- đổi

SRC_LANG = "en_XX"
TGT_LANG = "vi_VN"

tokenizer = AutoTokenizer.from_pretrained(HF_REPO_ID, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(HF_REPO_ID)

tokenizer.src_lang = SRC_LANG
tokenizer.tgt_lang = TGT_LANG

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("[OK] Loaded from hub:", HF_REPO_ID)
print("[INFO] device:", device)


special_tokens_map.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

[OK] Loaded from hub: ngothuyet/mbart50-envi
[INFO] device: cuda


Sử dụng beam search để dịch

In [18]:
import torch
from typing import List

MAX_SRC_LEN = 256

@torch.inference_mode()
def translate_en2vi(
    texts: List[str],
    num_beams: int = 5,
    length_penalty: float = 1.0,
    no_repeat_ngram_size: int = 3,
    max_new_tokens: int = 128,
):
    model.eval()
    tokenizer.src_lang = SRC_LANG
    tokenizer.tgt_lang = TGT_LANG

    enc = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_SRC_LEN
    ).to(model.device)

    generated = model.generate(
        **enc,
        forced_bos_token_id=tokenizer.lang_code_to_id[TGT_LANG],
        num_beams=num_beams,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        max_new_tokens=max_new_tokens,
        early_stopping=True,
    )

    return tokenizer.batch_decode(generated, skip_special_tokens=True)

# Demo nhanh
demo = [
    "It's necessary to provide health insurance communication and education for people who live in remote areas and participate interupted health insurance.",
    "I am learning natural language processing."
]
print("[INPUT ]", demo)
print("[OUTPUT]", translate_en2vi(demo, num_beams=5, length_penalty=1.0, no_repeat_ngram_size=3))


[INPUT ] ["It's necessary to provide health insurance communication and education for people who live in remote areas and participate interupted health insurance.", 'I am learning natural language processing.']
[OUTPUT] ['Cần thiết cung cấp dịch vụ truyền thông và giáo dục bảo hiểm y tế cho những người sống ở vùng xa xôi và tham gia BHYT.', 'Tôi đang học cách xử lý ngôn ngữ tự nhiên.']


Tính bleu trên toàn bộ tập test

In [19]:
from datasets import Dataset

# đọc song song public_test.en/vi -> list[(en, vi)]
test_pairs_raw = load_parallel(TEST_EN, TEST_VI, "test_raw")

# clean/filter/dedup giống pipeline train
test_pairs = clean_filter_dedup(test_pairs_raw, "test")

print(f"[INFO] public_test after clean: {len(test_pairs):,} pairs")
print("[SAMPLE TEST]", test_pairs[0])

# tạo ds_test (giữ nguyên ds train/val của anh nếu đã có)
test_ds = Dataset.from_dict({
    "src_text": [s for s, _ in test_pairs],
    "tgt_text": [t for _, t in test_pairs],
})

# gắn vào DatasetDict nếu đã có ds, còn không thì tạo ds mới
if "ds" in globals():
    ds["test"] = test_ds
else:
    from datasets import DatasetDict
    ds = DatasetDict({"test": test_ds})

print("[OK] ds['test'] size:", len(ds["test"]))


[INFO] test_raw lines -> 3,000 pairs.
[INFO] test: after clean+filter -> 2,946 pairs
[INFO] test: after dedup       -> 2,943 pairs
[INFO] public_test after clean: 2,943 pairs
[SAMPLE TEST] ('Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao', 'Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017')
[OK] ds['test'] size: 2943


In [20]:
import os, json, time
import torch
import sacrebleu
from tqdm.auto import tqdm

assert "ds" in globals() and "test" in ds, "Missing ds['test'] - run TEST-LOAD cell first."
assert "translate_en2vi" in globals(), "Missing translate_en2vi() - run inference cell first."
assert "OUTPUT_DIR" in globals(), "Missing OUTPUT_DIR (e.g., 'mbart50_envi')."
assert "trainer" in globals(), "Missing trainer - run training cell first (to push easily)."

def bleu_on_public_test_and_push(batch_size=16):
    srcs = ds["test"]["src_text"]
    refs = ds["test"]["tgt_text"]

    bs = batch_size if torch.cuda.is_available() else max(2, batch_size // 4)

    hyps = []
    t0 = time.time()

    for i in tqdm(range(0, len(srcs), bs), desc="BLEU on public_test", total=(len(srcs) + bs - 1)//bs):
        batch = srcs[i:i+bs]
        hyps.extend(
            translate_en2vi(
                batch,
                num_beams=5,
                length_penalty=1.0,
                no_repeat_ngram_size=3,
                max_new_tokens=128
            )
        )

    bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
    elapsed = time.time() - t0

    metrics = {
        "public_test_sacrebleu": float(bleu),
        "public_test_size": int(len(srcs)),
        "batch_size": int(bs),
        "elapsed_sec": float(elapsed),
    }

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    json_path = os.path.join(OUTPUT_DIR, "public_test_metrics.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)

    txt_path = os.path.join(OUTPUT_DIR, "public_test_bleu.txt")
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(f"public_test_sacrebleu\t{bleu:.4f}\n")

    print(f"\n[PUBLIC_TEST] sacreBLEU ({len(srcs)} samples): {bleu:.2f}")
    print("[OK] Saved:", json_path)
    print("[OK] Saved:", txt_path)

    # push lên repo model
    trainer.push_to_hub(commit_message=f"Add public_test sacreBLEU={bleu:.2f}")
    print("[OK] Pushed public_test BLEU files to Hugging Face.")

    return bleu, metrics

_ = bleu_on_public_test_and_push(batch_size=16)


BLEU on public_test:   0%|          | 0/184 [00:00<?, ?it/s]


[PUBLIC_TEST] sacreBLEU (2943 samples): 41.51
[OK] Saved: mbart50_envi/public_test_metrics.json
[OK] Saved: mbart50_envi/public_test_bleu.txt


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

[OK] Pushed public_test BLEU files to Hugging Face.


Hiển thị điểm bleu được lấy từ hugging face

In [21]:
import os, json

json_path = os.path.join(OUTPUT_DIR, "public_test_metrics.json")

with open(json_path, "r", encoding="utf-8") as f:
    metrics = json.load(f)

bleu = metrics["public_test_sacrebleu"]
n = metrics["public_test_size"]

print("="*60)
print(f"✅ sacreBLEU on public_test ({n} sentences): {bleu:.2f}")
print("="*60)


✅ sacreBLEU on public_test (2943 sentences): 41.51


Print 5 dòng test 

In [22]:
# kiểm tra điều kiện
assert "ds" in globals() and "test" in ds, "Missing ds['test']"
assert "translate_en2vi" in globals(), "Missing translate_en2vi()"

N_SHOW = 5  # số dòng muốn in

srcs = ds["test"]["src_text"][:N_SHOW]
tgts = ds["test"]["tgt_text"][:N_SHOW]

# dịch
preds = translate_en2vi(
    srcs,
    num_beams=5,
    length_penalty=1.0,
    no_repeat_ngram_size=3,
    max_new_tokens=128,
)

print("=" * 100)
for i, (src, tgt, pred) in enumerate(zip(srcs, tgts, preds), start=1):
    print(f"[{i}] SRC (EN): {src}")
    print(f"    TGT (VI): {tgt}")
    print(f"    PRED(VI): {pred}")
    print("-" * 100)
print("=" * 100)


[1] SRC (EN): Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao
    TGT (VI): Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017
    PRED(VI): Kiến thức, thực hành sử dụng dịch vụ y tế công cộng của người chủ thẻ BHYT và một số yếu tố ảnh hưởng ở Vientiane, Lao
----------------------------------------------------------------------------------------------------
[2] SRC (EN): Describe knowledge, practices in public health service utilization among health insurance card's holders and influencing factors in Vientiane, Lao PDR, 2017.
    TGT (VI): Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố liên quan tại tỉnh Viêng Chăn, Cộng hoà Dân chủ Nhân dân Lào năm 2017.
    P